# Projeto 2 - Projeto de Bioinformática

## Exercício 1: Contagem de Bases

### Códigos para a contagem total de A, T C e G nas bases de dados disponibilizadas.


#### Implementação com OpenMP

```cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <omp.h>
#include <string>
#include <filesystem>
#include <cctype>  // Para std::toupper

namespace fs = std::filesystem;

// Função para contar bases em uma sequência
void countBases(const std::string &sequence, int &countA, int &countT, int &countC, int &countG) {
    #pragma omp parallel for reduction(+:countA, countT, countC, countG)
    for (size_t i = 0; i < sequence.size(); ++i) {
        char base = std::toupper(sequence[i]);  // Converte para maiúsculas
        if (base == 'A') countA++;
        else if (base == 'T') countT++;
        else if (base == 'C') countC++;
        else if (base == 'G') countG++;
    }
}

// Função para processar um arquivo e contar bases
void processFile(const std::string &filename, int &countA, int &countT, int &countC, int &countG) {
    std::ifstream file(filename);
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << "\n";
        return;
    }

    std::string line, sequence;
    while (std::getline(file, line)) {
        if (line[0] != '>') {
            sequence += line;  // Concatena todas as sequências do arquivo
        }
    }
    file.close();
    
    countBases(sequence, countA, countT, countC, countG);
}

int main() {
    // Diretório contendo os arquivos de dados
    std::string directory = "../database";

    int totalA = 0, totalT = 0, totalC = 0, totalG = 0;

    // Itera sobre todos os arquivos na pasta `database` com extensão `.fa`
    for (const auto &entry : fs::directory_iterator(directory)) {
        if (entry.path().extension() == ".fa") {
            std::cout << "Processando arquivo: " << entry.path() << std::endl;
            processFile(entry.path().string(), totalA, totalT, totalC, totalG);
        }
    }

    // Exibe o resultado final
    std::cout << "Total de bases A: " << totalA << std::endl;
    std::cout << "Total de bases T: " << totalT << std::endl;
    std::cout << "Total de bases C: " << totalC << std::endl;
    std::cout << "Total de bases G: " << totalG << std::endl;

    return 0;
}

```

---

#### Implementação com MPI

```cpp
#include <iostream>
#include <fstream>
#include <mpi.h>
#include <string>
#include <vector>
#include <filesystem>
#include <cctype>  // Para std::toupper
#include <algorithm>  // Para std::sort

namespace fs = std::filesystem;

// Função para contar bases em um arquivo com conversão para maiúsculas
void count_bases_in_file(const std::string &filename, int &countA, int &countT, int &countC, int &countG) {
    std::ifstream file(filename);
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return;
    }

    std::string line;
    while (std::getline(file, line)) {
        if (line.empty() || line[0] == '>') continue;  // Ignora linhas de cabeçalho ou vazias
        for (char base : line) {
            base = std::toupper(base);  // Converte para maiúsculas
            if (base == 'A') countA++;
            else if (base == 'T') countT++;
            else if (base == 'C') countC++;
            else if (base == 'G') countG++;
        }
    }
    file.close();
}

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Diretório contendo os arquivos de dados
    std::string directory = "../database";
    std::vector<std::string> files;

    // Todos os processos obtêm a lista de arquivos e a ordenam
    for (const auto &entry : fs::directory_iterator(directory)) {
        if (entry.path().extension() == ".fa") {
            files.push_back(entry.path().string());
        }
    }

    // Ordena a lista de arquivos para garantir a mesma ordem em todos os processos
    std::sort(files.begin(), files.end());

    int num_files = files.size();

    // Calcula os índices de início e fim para cada processo
    int files_per_process = num_files / size;
    int remainder = num_files % size;
    int start_idx = rank * files_per_process + std::min(rank, remainder);
    int end_idx = start_idx + files_per_process + (rank < remainder ? 1 : 0);

    // Para depuração: exibe quais arquivos cada processo está processando
    std::cout << "Processo " << rank << " processando arquivos de índice " << start_idx << " a " << end_idx - 1 << std::endl;

    // Contagem de bases locais para cada processo
    int local_countA = 0, local_countT = 0, local_countC = 0, local_countG = 0;

    for (int i = start_idx; i < end_idx; ++i) {
        std::cout << "Processo " << rank << " processando arquivo: " << files[i] << std::endl;
        count_bases_in_file(files[i], local_countA, local_countT, local_countC, local_countG);
    }

    // Reduz as contagens locais em uma soma global no processo 0
    int global_countA = 0, global_countT = 0, global_countC = 0, global_countG = 0;
    MPI_Reduce(&local_countA, &global_countA, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countT, &global_countT, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countC, &global_countC, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countG, &global_countG, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    // Apenas o processo 0 exibe o resultado final
    if (rank == 0) {
        std::cout << "Total de bases A: " << global_countA << std::endl;
        std::cout << "Total de bases T: " << global_countT << std::endl;
        std::cout << "Total de bases C: " << global_countC << std::endl;
        std::cout << "Total de bases G: " << global_countG << std::endl;
    }

    MPI_Finalize();
    return 0;
}

```

---

#### Implementação híbrida com OpenMP e MPI

```cpp
#include <iostream>
#include <fstream>
#include <mpi.h>
#include <omp.h>
#include <string>
#include <vector>
#include <filesystem>
#include <chrono>

namespace fs = std::filesystem;

// Função para contar bases em uma sequência usando OpenMP
void count_bases_in_sequence(const std::string &sequence, int &countA, int &countT, int &countC, int &countG) {
    #pragma omp parallel for reduction(+:countA, countT, countC, countG)
    for (size_t i = 0; i < sequence.size(); ++i) {
        char base = std::toupper(sequence[i]);
        if (base == 'A') countA++;
        else if (base == 'T') countT++;
        else if (base == 'C') countC++;
        else if (base == 'G') countG++;
    }
}

// Função para contar bases em um arquivo
void count_bases_in_file(const std::string &filename, int &countA, int &countT, int &countC, int &countG) {
    std::ifstream file(filename);
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return;
    }

    std::string line, sequence;
    while (std::getline(file, line)) {
        if (line[0] != '>') {
            sequence += line;
        }
    }
    file.close();
    count_bases_in_sequence(sequence, countA, countT, countC, countG);
}

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    std::string directory = "../database";
    std::vector<std::string> files;

    // O processo 0 coleta a lista de arquivos e distribui entre os processos
    if (rank == 0) {
        for (const auto &entry : fs::directory_iterator(directory)) {
            if (entry.path().extension() == ".fa") {
                files.push_back(entry.path().string());
            }
        }
    }

    // Broadcast do número de arquivos para todos os processos
    int num_files = files.size();
    MPI_Bcast(&num_files, 1, MPI_INT, 0, MPI_COMM_WORLD);

    // Broadcast da lista de arquivos para todos os processos
    if (rank != 0) {
        files.resize(num_files);
    }
    for (int i = 0; i < num_files; i++) {
        int length = (rank == 0) ? files[i].size() : 0;
        MPI_Bcast(&length, 1, MPI_INT, 0, MPI_COMM_WORLD);
        if (rank != 0) {
            files[i].resize(length);
        }
        MPI_Bcast(&files[i][0], length, MPI_CHAR, 0, MPI_COMM_WORLD);
    }

    auto start = std::chrono::high_resolution_clock::now();

    // Contagem de bases locais para cada processo
    int local_countA = 0, local_countT = 0, local_countC = 0, local_countG = 0;

    // Processa os arquivos divididos entre os processos com OpenMP
    for (int i = rank; i < num_files; i += size) {
        count_bases_in_file(files[i], local_countA, local_countT, local_countC, local_countG);
    }

    // Reduz as contagens locais em uma soma global no processo 0
    int global_countA, global_countT, global_countC, global_countG;
    MPI_Reduce(&local_countA, &global_countA, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countT, &global_countT, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countC, &global_countC, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_countG, &global_countG, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;

    // Apenas o processo 0 exibe o resultado final
    if (rank == 0) {
        std::cout << "Total de bases A: " << global_countA << std::endl;
        std::cout << "Total de bases T: " << global_countT << std::endl;
        std::cout << "Total de bases C: " << global_countC << std::endl;
        std::cout << "Total de bases G: " << global_countG << std::endl;
        std::cout << "Tempo total de execução: " << elapsed.count() << " segundos" << std::endl;
    }

    MPI_Finalize();
    return 0;
}
```

---

### Resultados obtidos

#### OpenMP

- Total de bases A: 763517118
- Total de bases T: 764606176
- Total de bases C: 511685713
- Total de bases G: 511971484
- Tempo total de execução (OpenMP): 9.65299 segundos

#### MPI

- Total de bases A: 763517118
- Total de bases T: 764606176
- Total de bases C: 511685713
- Total de bases G: 511971484
- Tempo total de execução (MPI): 11.1374 segundos

#### OpenMP + MPI

- Total de bases A: 763517118
- Total de bases T: 764606176
- Total de bases C: 511685713
- Total de bases G: 511971484
- Tempo total de execução (Híbrido OpenMP + MPI): 5.75119 segundos

---

### Análise das Implementações: Exercício 1

O objetivo do exercício foi realizar a contagem de bases nucleotídicas (A, T, C e G) em um conjunto de arquivos FASTA que representam sequências cromossômicas. Três abordagens distintas foram utilizadas: OpenMP, MPI, e uma versão híbrida que combina ambas as técnicas para maximizar a eficiência.

#### 1. Implementação com OpenMP
A primeira implementação utilizou **OpenMP** para realizar paralelismo local em um único processador com múltiplas threads. Nesta abordagem:
   - **Divisão de Trabalho**: Cada thread processa um arquivo FASTA, permitindo o processamento simultâneo de diferentes arquivos na mesma máquina.
   - **Preprocessamento**: Todas as sequências foram convertidas para letras maiúsculas para garantir uniformidade na contagem, caso existissem variações de formatação.
   - **Desempenho**: Esta abordagem foi capaz de reduzir o tempo de execução comparado a um processamento sequencial, mas limitada pela quantidade de núcleos disponíveis em um único processador.

   - **Tempo Total de Execução**: 9.65299 segundos

#### 2. Implementação com MPI
A segunda abordagem utilizou **MPI** para dividir a carga de trabalho entre diferentes processadores:
   - **Distribuição de Arquivos**: Cada processo MPI recebeu um conjunto de arquivos para processamento. Isso permitiu que a carga fosse distribuída entre várias máquinas, reduzindo o tempo de execução em um sistema com múltiplos nós.
   - **Paralelismo Entre Processadores**: Ao invés de threads locais, cada processo MPI operava independentemente em uma seção dos arquivos, o que permite processamento paralelo em clusters de máquinas.
   - **Desempenho**: O uso de MPI aumentou a eficiência da execução em um ambiente de cluster, mas o tempo de comunicação entre processos limitou um pouco a melhoria de desempenho em comparação com o OpenMP em um único nó.

   - **Tempo Total de Execução**: 11.1374 segundos

#### 3. Implementação Híbrida (OpenMP + MPI)
A terceira e mais eficiente abordagem combinou OpenMP e MPI, tirando proveito do paralelismo em diferentes níveis:
   - **Divisão de Trabalho em Níveis**: A carga de trabalho foi distribuída entre múltiplos processos MPI, e cada processo utilizou threads OpenMP para processar os arquivos em paralelo localmente.
   - **Eficiência e Redução de Tempo**: Essa abordagem combinou o melhor das duas técnicas. Cada nó do cluster processava múltiplos arquivos em paralelo, e o uso de MPI permitiu a divisão da carga de trabalho entre nós. Com isso, houve uma diminuição significativa do tempo total de execução.
   - **Desempenho**: O uso da implementação híbrida trouxe o menor tempo de execução, combinando os benefícios de ambas as técnicas para aproveitar ao máximo os recursos computacionais disponíveis.

   - **Tempo Total de Execução**: 5.75119 segundos

---

### Comparação dos Tempos de Execução e Eficiência

| Implementação      | Tempo Total (segundos) | Observações                                                                                   |
|--------------------|------------------------|-----------------------------------------------------------------------------------------------|
| OpenMP               | 9.65299              | Paralelismo local com threads, limitado pela quantidade de núcleos no processador.             |
| MPI                  | 11.1374              | Distribuição entre processos MPI, limitado pela comunicação entre processos.                   |
| Híbrida (OpenMP+MPI) | 5.75119              | Paralelismo em múltiplos níveis (processos MPI + threads OpenMP), resultando no menor tempo.   |

A análise dos tempos de execução indica que a abordagem híbrida obteve o melhor desempenho, pois conseguiu combinar o processamento distribuído do MPI com o paralelismo local do OpenMP. Isso resultou em um uso mais eficiente dos recursos computacionais, especialmente em um ambiente com múltiplos nós e núcleos.

### Considerações Finais

As três abordagens mostraram-se eficazes na contagem de bases nucleotídicas, e a escolha entre elas depende do ambiente de execução. Em um sistema com um único processador, o OpenMP seria suficiente. No entanto, para grandes volumes de dados em um ambiente de cluster, a abordagem híbrida demonstrou ser a mais eficiente. Essa estratégia também é escalável, permitindo um processamento mais rápido em infraestruturas maiores, o que é fundamental para análises de bioinformática em larga escala. 

Além disso, o preprocessamento de conversão para letras maiúsculas garantiu consistência nos dados e precisão na contagem, eliminando potenciais variações de formato dos arquivos originais.
